In [1]:
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import torch

from nerfstudio.utils.eval_utils import eval_setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

Load trained model

In [2]:
#config, pipeline, checkpoint_path, _ = eval_setup(Path('/home/navlab-exxact/NeRF/nerfstudio/outputs/MoonTestScenario/regional-nerfacto/2023-10-13_135547/config.yml'))
config, pipeline, checkpoint_path, _ = eval_setup(Path('/home/navlab-exxact/NeRF/nerfstudio/outputs/GESMoonRender/regional-nerfacto/2023-10-25_144723/config.yml'))
#config, pipeline, checkpoint_path, _ = eval_setup(Path('/home/navlab-exxact/NeRF/nerfstudio/outputs/GESSanJose/regional-nerfacto/2023-10-24_135533/config.yml'))

[16:05:19] Auto image downscale factor of 1                                                 ]8;id=98199;file:///home/navlab-exxact/NeRF/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=828836;file:///home/navlab-exxact/NeRF/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#349\349]8;;\

Setting up training dataset...

Caching 600 out of 714 images, without resampling.

Setting up evaluation dataset...

Caching all 7 images.

Output()

tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 70. Fal

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/GESMoonRender/regional-nerfacto/2023-10-25_144723/nerfstudio_models/step-000029999.ckpt

Generate grid of xy points

In [18]:
N = 256
x, y = torch.meshgrid(
    torch.linspace(-2, 2, N, requires_grad=True, device=device),
    torch.linspace(-2, 2, N, requires_grad=True, device=device),
    indexing='ij'
)
XY_grid = torch.stack((x, y), dim=-1)
positions = XY_grid.reshape(-1, 2)

In [26]:
x = torch.rand(10,10)
y = torch.rand(10,10)
torch.stack((x,y), dim=-1).shape

torch.Size([10, 10, 2])

In [19]:
positions.shape

torch.Size([65536, 2])

Compute heights and plot heatmap

In [20]:
heights = pipeline.model.field.positions_to_heights(positions)

In [21]:
def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [22]:
h_x = gradient(heights, x)
h_y = gradient(heights, y)

In [23]:
torch.max(h_x)

tensor(62.9042, device='cuda:0', grad_fn=<MaxBackward1>)

In [ ]:
heights = pipeline.model.field.positions_to_heights(positions)

# Remove outliers



fig = px.imshow(heights.reshape(N, N).detach().cpu().numpy())

# fig = px.imshow(gradient(heights, x).detach().cpu().numpy())
fig.show()